In [ ]:
#week3 linear data 
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from IPython.display import clear_output

def main():
   adc = 0
   adc = float(input("please enter ADC observed:"))
   clear_output(wait=True)  # Clear previous output
   calculation(adc)

def calculation(ADCin):
   water = []
   ADC = []

   file1 = open("recordValue3.csv", "r")
   for row in file1:
      data = row.strip().split(",")
      water.append(float(data[0]))
      ADC.append(float(data[1]))
   file1.close()

   waterx = np.array(water, dtype=int)
   ADCY = np.array(ADC, dtype=float)
   slope, intercept = np.polyfit(waterx, ADCY,1) 
   ADCy = slope*waterx+intercept

   mindiff = 1e10
   input2ymindiff = 1e10
   mindiffindex = 0
   for i in range(len(ADCY)):
      if (np.abs(ADCin - ADCY[i]) < np.abs(mindiff)):
         mindiff = ADCin - ADCY[i]
         input2ymindiff = ADCin - ADCy[i]
         mindiffindex = i

   if input2ymindiff >= 0:
      output = ADCin - input2ymindiff
   elif input2ymindiff < 0:
      output = ADCin + (-input2ymindiff)

   #calculate maxdiff from raw data to linear line
   maxdiff = 0
   diffp = 0
   maxdiffindex = 0
   for i in range(len(ADCY)):
      if (np.abs(ADCy[i] - ADCY[i]) > maxdiff):
         maxdiff = np.abs(ADCy[i] - ADCY[i])
         diffp = maxdiff / (np.max(ADCY) - np.min(ADCY)) * 100
         maxdiffindex = i
   print(f"max diff between raw data and line at {waterx[maxdiffindex]},{diffp:.2f}error ")

   #calculate R2
   SSE = sum((ADCy-ADCY)**2)
   print(f"SSE ={SSE}")

   SST = sum((ADCy-np.mean(ADCy))**2)
   print(f"SST ={SST}")

   R2 = 1 - (SSE / SST)
   print(f"R2 ={R2}")

   #calculate non-linearity
   # m = (min(ADCY) - max(ADCY))/(waterx[-1] - waterx[0])
   # c = ADCY[0]
   # terminalLine = m*waterx + c
   # deviation = abs(ADCY - ADCy)
   # nonlinearity = (max(deviation)/(max(ADCY)-min(ADCY))) * 100
   # print(f"non-linearity = {nonlinearity:.4f}")
   #plt.plot(waterx,terminalLine,label="terminal point line")

   plt.figure(1)
   plt.scatter(water,ADC,label = "Raw Data")
   # plt.axvline(x=40, color='r')
   plt.plot(waterx,ADCy,label="Linear Transfer Function")
   plt.title(f"ADC v water, ADC={slope:.2f}*water + {intercept:.2f}")
   plt.scatter(waterx[mindiffindex],output, label="Pred point")

   plt.xticks(np.arange(0, 42 , 2))
   plt.legend()
   plt.grid()
   plt.xlabel("Water content(ml)")
   plt.ylabel("ADC value")
   plt.tight_layout()
   plt.show()

   plt.figure(2)
   #calculate moisture%
   x = waterx
   rawy = (ADCY - np.max(ADCY)) / (np.min(ADCY) - np.max(ADCY)) * 100
   y = (ADCy - np.max(ADCy)) / (np.min(ADCy) - np.max(ADCy)) * 100
   slope = (y[-1] - y[0]) / (x[-1] - x[0])
   inter = y[0]
   # y10 = y + 10
   # yn10 = y - 10
   
   #calculate non-linearity
   deviation = np.abs(y - rawy)
   nonlinearity = (max(deviation)/(max(y)-min(y))) * 100
   print(f"non-linearity = {nonlinearity:.4f}")
   #calculate maxdiff from raw data to linear line
   maxdiff = 0
   diffp = 0
   maxdiffindex = 0
   for i in range(len(rawy)):
      if (np.abs(y[i] - rawy[i]) > maxdiff):
         maxdiff = np.abs(y[i] - rawy[i])
         diffp = maxdiff / (np.max(y) - np.min(y)) * 100
         maxdiffindex = i
   print(f"max diff between raw data and line at {waterx[maxdiffindex]},{diffp:.2f}error ")

   plt.scatter(x,rawy,label="converted raw data")
   plt.plot(x,y,label="transfer function")
   # plt.plot(x,y10,label="upper 10%")
   # plt.plot(x,yn10,label="lower 10%")
   #plt.scatter(waterx[mindiffindex],y[mindiffindex],label="pred point")

   plt.grid()
   plt.xticks(np.arange(0, 42 , 2))
   plt.yticks(np.arange(0, 102 , 10))
   plt.title(f"moisture% v water content, m = {slope}*water + {inter}")
   plt.xlabel("Water content(ml)")
   plt.ylabel("Moisture level(%)")
   plt.legend()
   plt.tight_layout()
   plt.show()

   if ADCin > min(ADCY):
      print(f"actual ADC:{ADCin}")
      print(f"predicted ADC:{output}")
      actualContent = (ADCin - np.max(ADCY)) / (np.min(ADCY) - np.max(ADCY)) * 100
      if actualContent <= 0:
         actualContent = 0
      print(f"actual water content:{actualContent:.2f}%")
      print(f"predicted water content:{waterx[mindiffindex]}ml,{y[mindiffindex]:.2f}% wet")
   else:
      print("Soil is wet (>40ml)")

   main() 

if __name__ == "__main__":
    main()

In [ ]:
file1 = open("recordValue2 - Sheet1.csv", "r")
water = []
voltage = []
for row in file1:
   data = row.strip().split(",")
   water.append(float(data[0]))
   voltage.append(float(data[2]))
file1.close()
ADC = []

file2 = open("recordValue3.csv", "r")
for row in file2:
   data = row.strip().split(",")
   ADC.append(float(data[1]))
file2.close()

waterx = np.array(water, dtype=int)
ADCY = np.array(ADC, dtype=float)
voltageY = np.array(voltage, dtype=float)
slope, intercept = np.polyfit(waterx, ADCY,1) 
ADCy = slope*waterx+intercept
m, c = np.polyfit(waterx, voltageY,1) 

plt.figure(figsize=(10, 6))
plt.subplot(1,2,1)
#plot voltage
plt.scatter(water,voltage,label="raw data")
plt.plot(waterx,m*waterx+c,label="linear trend")
plt.title(f"voltage v water, V = {m:.2f}*water + {c:.2f}")

plt.xticks(np.arange(0, 42 , 2))
plt.xlabel("Water content(ml)")
plt.ylabel("Voltage(V)")
plt.legend()
plt.grid()


plt.subplot(1,2,2)
plt.scatter(water,ADC,label = "Raw Data")
plt.plot(waterx,ADCy,label="Linear Transfer Function")
plt.title(f"ADC v water, ADC={slope:.2f}*water + {intercept:.2f}")

plt.xticks(np.arange(0, 42 , 2))
plt.xlabel("Water content(ml)")
plt.ylabel("ADC value")
plt.tight_layout()
plt.legend()
plt.grid()
plt.show()

In [ ]:
#calculate repeatability
file2 = open("recordValue3 - Sheet1.csv", "r")
out = []
for row in file2:
   output = []
   data = row.strip().split(",")
   water.append(float(data[0]))
   output.append(float(data[1]))
   output.append(float(data[2]))
   output.append(float(data[3]))
   out.append(output)
file2.close()

water = np.array(water, dtype=int)
out = np.array(out, dtype=float)

repeatability = []

max_out = np.max(out, axis=1)
min_out = np.min(out, axis=1)
range_out = max(max_out)-min(min_out)
for r in range(len(out)):
    maxdiff = abs(max(out[r]) - min(out[r]))
    repeatability.append((maxdiff / range_out) * 100)

print(f"repeatability = {np.mean(repeatability):.4f}")

In [ ]:
#recordValue1
# 52,722.28,2.33
# 54,700.33,2.329
# 56,654.14,2.3
# 58,608.33,2.295
# 60,584,2.256
# 62,587,2.31
# 64,561.8,2.29
# 66,520.05,2.279
# 68,487.91,2.268

#recordValue2
# 42,1035.89,2.478
# 44,886.3,2.428
# 46,878,2.425
# 48,797.8,2.415
# 50,712.56,2.385
# 52,756.5,2.36
# 54,706.1,2.378
# 56,650.4,2.399
# 58,638.73,2.376
# 60,539.56,2.329
# 62,562.7,2.34
# 64,555.6,2.323
# 66,455.7,2.3
# 68,423,2.3
# 70,506.89,2.31
# 72,432.22,2.294
# 74,435.2,2.304
# 76,393.6,2.29
# 78,453.8,2.23
# 80,460.2,2.31

#recordValue3
# 20,2129.727273
# 42,912.0909091
# 44,908.1969697
# 46,865.7727273
# 48,855.4469697
# 50,845.7424242
# 52,786.4545455
# 54,773.3484848
# 56,732.7424242
# 58,665.0606061
# 60,632.780303
# 62,664.9318182
# 64,597.0681818
# 66,595.719697
# 68,626.1363636
# 70,561.8106061